In [1]:
import pandas as pd

In [2]:
# Load the first CSV file
df1 = pd.read_csv('NHRPsankalpUpdated-NotificationData_DATA_LABELS_2025-08-19_0843.csv')

In [3]:
# Load the second CSV file
df2 = pd.read_csv('BirthNotificationBy Asha_2025-08-19.csv')

In [4]:
# Display the first few rows and information about the first DataFrame
print("DataFrame 1 - NHRPsankalpUpdated-NotificationData_DATA_LABELS_2025-08-19_0843.csv")
print(df1.head())
print(df1.info())

DataFrame 1 - NHRPsankalpUpdated-NotificationData_DATA_LABELS_2025-08-19_0843.csv
  Record ID Q1. District Name             Q2. Data Collected From  \
0     112-1           varansi  Delivery Register (Facility Visit)   
1     112-2           varansi  Delivery Register (Facility Visit)   
2     112-3           varansi  Delivery Register (Facility Visit)   
3     112-4           varansi  Delivery Register (Facility Visit)   
4     112-5           varansi  Delivery Register (Facility Visit)   

  Q3. Name of the facility where the data was collected    \
0                                        AAM Rajwari        
1                                         PHC Pindra        
2                                     CHC Narpat Pur        
3                                     CHC Narpat Pur        
4                                     CHC Narpat Pur        

   Q4. Other Facility Name Q7. Call/Visit Date  Call Hour  Call Minute  \
0                      NaN          05-03-2025          5     

In [5]:
# Display the first few rows and information about the second DataFrame
print("DataFrame 2 - BirthNotificationBy Asha_2025-08-19.csv")
print(df2.head())
print(df2.info())

DataFrame 2 - BirthNotificationBy Asha_2025-08-19.csv
   Q1.RecordID         Event Name Q1. Data Collected From  \
0         6949  BirthNotification     Asha (through call)   
1         6950  BirthNotification     Asha (through call)   
2         6951  BirthNotification     Asha (through call)   
3         6952  BirthNotification     Asha (through call)   
4         6953  BirthNotification     Asha (through call)   

   Q2. Name of the facility where the data was collected    \
0                                                NaN         
1                                                NaN         
2                                                NaN         
3                                                NaN         
4                                                NaN         

   Q2.1 Other Facility Name Q3. Call/Visit Date  Call Hour  Call Minute  \
0                       NaN          25-07-2025         12           45   
1                       NaN          25-07-2025         

In [6]:
# Clean column names by stripping whitespace
df1.columns = df1.columns.str.strip()
df2.columns = df2.columns.str.strip()

In [7]:
# Define meaningful combinations of columns for finding duplicates
combinations_to_check = [
    {
        'name': 'Mother Father Name, Mobile, Delivery Date, and Village',
        'df1_cols': ['Q14. Name of Mother', 'Q15. Name of Father', 'Q16. Family Mobile Number', 'Q33. Delivery Date', 'Village Name'],
        'df2_cols': ['Q9. Name of Mother', 'Q10. Name of Father', 'Q11. Family Mobile Number', 'Q24. Delivery Date', 'Q17. VillageName']
    },
    {
        'name': 'Mother Father Name, Mobile and Delivery Date',
        'df1_cols': ['Q14. Name of Mother', 'Q15. Name of Father', 'Q16. Family Mobile Number', 'Q33. Delivery Date'],
        'df2_cols': ['Q9. Name of Mother', 'Q10. Name of Father', 'Q11. Family Mobile Number', 'Q24. Delivery Date']
    },
    {
        'name': 'Mother Father Name and Mobile',
        'df1_cols': ['Q14. Name of Mother', 'Q15. Name of Father', 'Q16. Family Mobile Number'],
        'df2_cols': ['Q9. Name of Mother', 'Q10. Name of Father', 'Q11. Family Mobile Number']
    },
    {
        'name': 'Mother Father Name and Delivery Date',
        'df1_cols': ['Q14. Name of Mother', 'Q15. Name of Father', 'Q33. Delivery Date'],
        'df2_cols': ['Q9. Name of Mother', 'Q10. Name of Father', 'Q24. Delivery Date']
    },
    {
        'name': 'Mobile Number and Delivery Date',
        'df1_cols': ['Q16. Family Mobile Number', 'Q33. Delivery Date'],
        'df2_cols': ['Q11. Family Mobile Number', 'Q24. Delivery Date']
    }
]

In [8]:
# List to hold all duplicate dataframes for later concatenation
all_duplicates_list = []

In [9]:
# Loop through each combination and find duplicates
for i, combo in enumerate(combinations_to_check):
    print(f"Processing combination {i+1}: {combo['name']}")

    # Make copies to work with inside the loop
    df1_temp = df1.copy()
    df2_temp = df2.copy()

    # Standardize data types for a cleaner merge
    for col in combo['df1_cols']:
        if 'Mobile Number' in col:
            df1_temp[col] = df1_temp[col].astype(str)
        elif 'Date' in col:
            df1_temp[col] = pd.to_datetime(df1_temp[col], errors='coerce')

    for col in combo['df2_cols']:
        if 'Mobile Number' in col:
            df2_temp[col] = df2_temp[col].astype(str)
        elif 'Date' in col:
            df2_temp[col] = pd.to_datetime(df2_temp[col], errors='coerce')

    # Perform an inner merge to find common rows
    merged_duplicates = pd.merge(df1_temp, df2_temp, how='inner', left_on=combo['df1_cols'], right_on=combo['df2_cols'], suffixes=('_df1', '_df2'))

    # Save the individual duplicate file
    output_filename = f"duplicates_combo_{i+1}.csv"
    merged_duplicates.to_csv(output_filename, index=False)
    print(f"  Found {len(merged_duplicates)} duplicate rows. Saved to '{output_filename}'")

     # Add the current merged DataFrame to the master list
    all_duplicates_list.append(merged_duplicates)

Processing combination 1: Mother Father Name, Mobile, Delivery Date, and Village
  Found 0 duplicate rows. Saved to 'duplicates_combo_1.csv'
Processing combination 2: Mother Father Name, Mobile and Delivery Date
  Found 2 duplicate rows. Saved to 'duplicates_combo_2.csv'
Processing combination 3: Mother Father Name and Mobile
  Found 3 duplicate rows. Saved to 'duplicates_combo_3.csv'
Processing combination 4: Mother Father Name and Delivery Date
  Found 12 duplicate rows. Saved to 'duplicates_combo_4.csv'
Processing combination 5: Mobile Number and Delivery Date
  Found 476 duplicate rows. Saved to 'duplicates_combo_5.csv'


In [11]:
# Concatenate all the individual duplicate dataframes and remove duplicates
if all_duplicates_list:
    final_all_duplicates = pd.concat(all_duplicates_list, ignore_index=True)
    
    # Drop duplicates based on the most robust key combination
    cols_to_drop_on = ['Q14. Name of Mother', 'Q15. Name of Father', 'Q16. Family Mobile Number', 'Q33. Delivery Date']
    final_all_duplicates.drop_duplicates(subset=cols_to_drop_on, inplace=True)
    
    # Save the final file with all unique duplicates
    final_output_filename = "all_unique_duplicates.csv"
    final_all_duplicates.to_csv(final_output_filename, index=False)
    print(f"\nTotal unique duplicates across all combinations found: {len(final_all_duplicates)}. Saved to '{final_output_filename}'")
else:
    print("\nNo duplicates found in any combination.")


Total unique duplicates across all combinations found: 150. Saved to 'all_unique_duplicates.csv'
